# 部署和使用ChatGLM进行中文问答

### SageMaker  Endpoint 部署ChatGLM
  
[ChatGLM](https://github.com/THUDM/ChatGLM-6B): ChatGLM-6B 是一个开源的、支持中英双语的对话语言模型，基于 General Language Model (GLM) 架构，具有 62 亿参数。结合模型量化技术，用户可以在消费级的显卡上进行本地部署（INT4 量化级别下最低只需 6GB 显存）。 ChatGLM-6B 使用了和 ChatGPT 相似的技术，针对中文问答和对话进行了优化。经过约 1T 标识符的中英双语训练，辅以监督微调、反馈自助、人类反馈强化学习等技术的加持，62 亿参数的 ChatGLM-6B 已经能生成相当符合人类偏好的回答。

首先，我们使用Sagemaker部署ChatGLM模型。
#### 准备
1. 升级boto3, sagemaker python sdk  
2. 准备inference.py, requirements.txt

In [2]:
# 如果需要，更新sagemaker和 aws python sdk boto3
# !pip install --upgrade boto3
# !pip install --upgrade sagemaker

In [3]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

接下来，我们使用Sagemaker进行模型部署。

In [4]:
from sagemaker.pytorch.model import PyTorchModel

model_name = None
entry_point = 'inference-chatglm.py'
framework_version = '1.13.1'
py_version = 'py39'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

!touch dummy
!tar czvf model.tar.gz dummy
!rm -f dummy

model = PyTorchModel(
    name = model_name,
    model_data = "./model.tar.gz",
    entry_point = entry_point,
    source_dir = './code-chatglm',
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

dummy


我们使用了bring your own script 的方式部署模型，通过提供一个dummy的模型文件（model.tar.gz），在 *code/inference-chatglm.py* python脚本里面，部署了模型：
```python
def model_fn(model_dir):
    model = AutoModel.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True).half().cuda()
    return model
```


然后部署该模型为 Sagemaker endpoint

In [5]:

# deploy as sagemaker endpoint
endpoint_name = 'mt-chatglm-6b-entpoint'
instance_type = 'ml.g4dn.2xlarge'
instance_count = 1

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

--------!

In [6]:
inputs= [
    {"ask": "写一首关于交通信号灯的诗"},
    {"ask": "陨石为什么总能落在陨石坑里?" },
    {"ask": "为什么爸妈结婚没叫我参加婚礼?"}
]
response = predictor.predict(inputs[0])
print("\n\n问题: ", inputs[0]["ask"], "\n回答:\n", response["answer"])
response = predictor.predict(inputs[1])
print("\n\n问题: ", inputs[1]["ask"], "\n回答:\n", response["answer"])
response = predictor.predict(inputs[2])
print("\n\n问题: ", inputs[2]["ask"], "\n回答:\n", response["answer"])



问题:  写一首关于交通信号灯的诗 
回答:
 交通信号灯，明明确确
指引着我们前行的方向
红绿黄三种颜色，
象征着生命与死亡的距离

当红灯亮时，停车等待
当绿灯亮时，继续前进
黄灯闪烁，提醒我们注意
以免发生不必要的事故

交通信号灯，是我们的指引
让我们遵守交通规则
不随意穿梭，不酒后驾车
让我们的安全之路，更加畅通

交通信号灯，是交通的守护者
让我们感受到了生命的可贵
让我们学会了尊重，学会了珍惜

所以，我们要永远遵守交通信号灯
让道路变得更加安全，更加畅通
让生命在交通信号灯的指引下，更加精彩


问题:  陨石为什么总能落在陨石坑里? 
回答:
 陨石坑是因为陨石撞击地球表面而形成的环形凹坑，陨石坑内部通常会有一些水坑和岩石碎片，这是因为陨石在进入地球大气层时，由于与大气分子的摩擦而产生高温和高压，导致陨石表面发生熔融和烧蚀。

陨石坑内部由于陨石与岩石的接触，会产生巨大的压力，导致陨石表面发生变形和破裂，同时陨石坑内部也会产生应力集中，使得陨石更容易被压坏或烧蚀。

另外，陨石坑内部通常会有一些地形变化和水流，这些地形变化和水流也会对陨石产生影响，使得陨石更容易被吸引到坑内。

因此，陨石落在坑内是由于陨石与地球表面之间的相互作用，使得陨石表面发生变形和破裂，同时陨石坑内部也会产生应力集中，使得陨石更容易被吸引到坑内。


问题:  为什么爸妈结婚没叫我参加婚礼? 
回答:
 父母结婚时没叫我参加婚礼，可能是因为他们认为我年龄太小，还不适合参加这样的正式场合；也可能是他们认为我的婚礼不会影响到他们的生活，所以他们就没有叫我参加。

父母的想法和感受都是正常的，可能是因为他们考虑到我的年龄和成长环境等因素。无论原因是什么，我都应该尊重他们的决定，并且理解他们的想法。


### ChatGLM 测试
下面，我们使用真格资金建立的一套中文语言模型的测试集，来对这个模型进行测试：
https://github.com/zhenbench/z-bench

真格基金提供了3个级别的测试集，基础能力、进阶能力和垂直能力，并且提供了使用 GPT3.5 和 GPT4 的参考答案，可以用这个测试集进行问答测试，并跟GPT的结果进行比较，来检验语言模型的能力。

#### 基础能力
首先来看看基础能力：

In [24]:
import pandas as pd
benchdf_common = pd.read_csv("./z-bench/common.samples.csv")

In [27]:
benchdf_common.shape

(63, 5)

In [40]:
import random
r = random.randint(0, 63)
test_data = benchdf_common.iloc[r]

response = predictor.predict({"ask": test_data["Prompt"], "params": {"temperature": 0.99}})
print("任务类型:", test_data["任务类型"], "参考回答:", test_data["参考回答"])
print("Prompt:", test_data["Prompt"])
print("ChatGLM:", response["answer"])
print("GPT3.5:", test_data["GPT3.5"])
print("GPT4:", test_data["GPT4"])

任务类型: `语义理解` 参考回答: nan
Prompt: 你如何理解这首诗：今天的紫菜汤有点咸了/但紫菜却不在意/飘飘荡荡/好像回到了大海
ChatGLM: 这首诗可能是一首关于紫菜的民谣，描述了紫菜在海水中自由自在地飘荡，似乎不受任何限制，同时也表达了作者对紫菜这种海洋生物的尊重和欣赏。

诗中的“紫菜”可能是指海洋中的一些小型海洋生物，如紫菜、海龙珠等，这些生物通常在海水中漂浮、生长和繁殖。它们没有像其他生物那样受到人类的干扰和限制，而是自由自在地享受着海洋带来的美好。

诗人用“有点咸了”来形容紫菜汤的味道，这可能暗示着紫菜对于这些人类所制造的食物的不喜欢或不适应，但是它并没有因此而感到沮丧或失落，而是仍然保持着自己的本性和自由。

整首诗表达了一种对自然和自由的向往和尊重，以及对人类所创造的美好事物的感激和欣赏。
GPT3.5: 这首诗可以从不同角度进行理解，以下是一种可能的解读方式：这首诗以"紫菜汤"为主题，但其实它所传递的是一种对生命、对归属感的思考。诗人发现今天的紫菜汤有点咸了，但紫菜却不在意，这描绘了紫菜作为一种海洋生物的本质，它并不在乎周围的变化，而是顺应大海的潮汐和韵律。接着，诗人通过"飘飘荡荡"的描写表达了自己仿佛也回到了大海的感受，这种感觉或许是因为他对紫菜、对海洋的感受与情感，让他产生了一种强烈的归属感和认同感。整首诗所表现出的情感，或许是诗人对自己身份认同的探索。他或许感到自己与社会环境的不协调和异化，但是在对大自然的感悟中，他找到了一种超越个人的、更广阔的归属感。
GPT4: 这首诗表达了一种对生活琐事的微妙反思。通过描述今天的紫菜汤有点咸，引发了对紫菜本身的关注。紫菜在诗中并不在意汤的咸淡，反而在飘荡中回到了大海的意境，好像它回到了自己的根源和起点。诗人可能想传达一种超脱的心境，提醒人们不要过于在意生活中的琐碎事物，而是要像紫菜一样，随波逐流，与自己的内心和本质保持联系。这样的态度有助于让我们更好地面对生活中的起起伏伏，保持一颗平和的心。


#### 进阶能力

In [4]:
import pandas as pd
benchdf_emergent = pd.read_csv("./z-bench/emergent.samples.csv")

In [5]:
benchdf_emergent.head()

,Prompt,任务类型,参考回答,GPT3.5,GPT4
0,如果我在空中垂直抛球，球最初向哪个方向行进？,`物理直觉`,向上,当你在空中垂直抛球时，球最初行进的方向是向上的，也就是和地面垂直的方向。在球到达最高点时，它...,当你在空中垂直向上抛球时，球最初的方向是向上。它将沿着垂直方向受到重力的作用而减速，直至在其...
1,金元素属于哪一种化学键？,`物理直觉`,金属键,金是一种金属元素，因此通常会形成金属键。金属键是由金属元素组成的化学键，这些金属元素的原子之...,金属元素，如金（Au），通常与其他金属元素通过金属键（metallic bond）相互连接。...
2,假设一个原子发射一个光子，原子会发生什么？,`物理直觉`,失去能量,当一个原子发射一个光子时，它会处于一个激发态或高能态，并且会向低能态或基态释放能量。在这个过...,当一个原子发射一个光子时，这意味着原子中的一个电子从一个较高的能级跃迁到一个较低的能级。在这...
3,假设一辆车可以在 3.85s 的时间内从 0 加速到 27.8 m/s，请计算这辆车的加速度...,`物理`,7.22 m/s/s,根据运动学公式，加速度等于速度变化量除以时间。那么，这辆车的加速度为：a = (27.8 m...,为了计算加速度，我们可以使用以下公式：加速度（a）= (最终速度（vf） - 初始速度（vi...
4,Zdeno Chara 以 175.1 km/h 的速度创造了最快的冰球拍击记录。假设一个冰...,`物理`,a) 8.28 kg·m/s b) Chara 的拍击速度为 175.1 km/h = 48...,a) Chara 的拍击球的动量为：p = mv其中，m = 0.17 kg 是冰球的质量，...,a) 要计算冰球的动量，我们使用动量公式：p = m * v，其中 p 为动量，m 为质量，...


In [20]:
import random
r = random.randint(0, 209)
# test_data = benchdf_emergent.iloc[r]

response = predictor.predict({"ask": test_data["Prompt"]})
print("任务类型:", test_data["任务类型"], "参考回答:", test_data["参考回答"])
print("Prompt:", test_data["Prompt"])
print("ChatGLM:", response["answer"])
print("GPT3.5:", test_data["GPT3.5"])
print("GPT4:", test_data["GPT4"])

任务类型: `物理直觉` 参考回答: 金属键
Prompt: 金元素属于哪一种化学键？
ChatGLM: 金元素属于离子键。金属金的原子结构中，金属键是金属原子之间形成的化学键，使金原子结合在一起。金属键是由金属原子的电子构型和外部电场等因素决定的，它们的特点是强、稳定，并且不受温度和压力的影响。
GPT3.5: 金是一种金属元素，因此通常会形成金属键。金属键是由金属元素组成的化学键，这些金属元素的原子之间共享其外层电子，形成一个具有高电导率和高热导率的三维电子海。
GPT4: 金属元素，如金（Au），通常与其他金属元素通过金属键（metallic bond）相互连接。金属键是一种特殊的化学键，其中金属原子之间的价电子在整个晶格中自由流动，形成“电子海”。这种电子分布使得金属具有良好的导电性和导热性。因此，金元素属于金属键类型。


### 垂直类的问题

In [31]:
import pandas as pd
benchdf_specialized = pd.read_csv("./z-bench/specialized.samples.csv")
benchdf_specialized.shape

(39, 5)

In [33]:
import random
r = random.randint(0, 39)
test_data = benchdf_specialized.iloc[r]

response = predictor.predict({"ask": test_data["Prompt"]})
print("任务类型:", test_data["任务类型"], "参考回答:", test_data["参考回答"])
print("Prompt:", test_data["Prompt"])
print("ChatGLM:", response["answer"])
print("GPT3.5:", test_data["GPT3.5"])
print("GPT4:", test_data["GPT4"])

任务类型: `医学` 参考回答: 80%
Prompt: 截至2017年，全球1岁的儿童中有多少已接种某种疾病的疫苗？
ChatGLM: 很抱歉，我无法提供截至2017年全球1岁的儿童中已接种某种疾病的疫苗的数据。因为我的知识截止日期是2021年，而且我无法访问实时数据。

不过，疫苗接种率是一个全球性的公共卫生问题，许多组织和机构都会定期发布有关全球疫苗接种率的数据。您可以搜索相关的报告或数据，以了解全球疫苗接种率的情况。
GPT3.5: 据世界卫生组织（WHO）统计，截至2017年，全球有约89%的1岁儿童已接种了至少一种疾病的疫苗。这是一个令人鼓舞的数字，表明了疫苗接种在预防疾病方面的重要性和成功性。
GPT4: 截至2017年，全球约有85%的1岁儿童接种了某种疾病的疫苗。这一数据主要是针对三剂联合疫苗（包括百日咳、白喉和破伤风疫苗）的全球覆盖率。然而，请注意，这一比例可能因地区和国家的不同而有所差异。在某些国家和地区，疫苗接种率可能高于这个数字，在另一些国家和地区，疫苗接种率可能较低。


### 通过Sagemaker Endpoint调用
我们已经将模型部署到了Sagemaker Endpoint上，我们就可以通过这个Endpoint名称，来调用模型进行推理，这样即使你停止了这个notebook，也能使用已经部署的模型。

In [1]:
import json
import boto3

client = boto3.client('runtime.sagemaker')

def query_endpoint(encoded_json):
    response = client.invoke_endpoint(EndpointName="mt-chatglm-6b-entpoint", ContentType='application/json', Body=encoded_json)
    model_predictions = json.loads(response['Body'].read())
    generated_text = model_predictions["answer"]
    return generated_text

In [2]:
payload = {"ask": "信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案："}
resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

阿根廷，梅西。


### 删除Endpoint
如果部署的模型已经不使用，可以删除 Endpoint，避免不必要的开支。

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()